In [ ]:
import subprocess

# Uninstall gradio cleanly first
subprocess.run(['pip', 'uninstall', '-y', 'gradio'], check=True)
# Uninstall huggingface-hub to ensure the correct compatible version is installed
subprocess.run(['pip', 'uninstall', '-y', 'huggingface-hub'], check=False)

# Then install specific stable version of gradio along with other packages
# and explicitly install a compatible huggingface-hub version
!pip install --upgrade pillow reportlab gradio==4.28.0 huggingface-hub==0.19.4

  Using cached pillow-12.1.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
  Using cached gradio-4.28.0-py3-none-any.whl.metadata (15 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
Using cached gradio-4.28.0-py3-none-any.whl (12.2 MB)
Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.2 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.19.4 which is incompatible.
peft 0.18.1 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.19.4 which is incompatible.
accelerate 1.12.0 requires huggingface_hub>=0.21.0, but you have huggingface-hub 0.19.4 which is incompatible.
diffusers 0.36.0 requires huggingface-hub<2.0,>=0.34.0, but you have huggingface-hub 0.19.4 which is incompatible.
transformers 5.

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
import tempfile
import os
import gradio as gr

# A4 at 300 DPI
A4_PORTRAIT = (2480, 3508)
A4_LANDSCAPE = (3508, 2480)

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Load a clean modern font if available
try:
    FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
    TITLE_FONT = ImageFont.truetype(FONT_PATH, 90)
    SUBTITLE_FONT = ImageFont.truetype(FONT_PATH, 55)
    BODY_FONT = ImageFont.truetype(FONT_PATH, 38)
except:
    TITLE_FONT = ImageFont.load_default()
    SUBTITLE_FONT = ImageFont.load_default()
    BODY_FONT = ImageFont.load_default()

In [ ]:
def fit_image(img, max_w, max_h):
    """Resize while preserving aspect ratio."""
    w, h = img.size
    scale = min(max_w / w, max_h / h)
    new_size = (int(w * scale), int(h * scale))
    return img.resize(new_size, Image.Resampling.LANCZOS)


def wrap_text(text, font, max_width):
    """Smart text wrapping using font metrics."""
    if not text:
        return []

    words = text.split()
    lines = []
    current = ""

    for word in words:
        test = current + " " + word if current else word
        if font.getlength(test) <= max_width:
            current = test
        else:
            lines.append(current)
            current = word

    if current:
        lines.append(current)

    return lines


def draw_text_block(draw, text, font, x, y, max_width, max_height, line_spacing=1.35):
    """Draw wrapped text inside a bounding box."""
    if not text:
        return

    lines = wrap_text(text, font, max_width)
    line_height = int(font.size * line_spacing)

    for line in lines:
        if y + line_height > max_height:
            break
        draw.text((x, y), line, fill=BLACK, font=font)
        y += line_height

In [ ]:
def layout_grid(images, title, subtitle, body, page_size):
    W, H = page_size
    page = Image.new("RGB", page_size, WHITE)
    draw = ImageDraw.Draw(page)

    margin = int(W * 0.06)
    gutter = int(W * 0.03)

    # Text blocks
    title_y = margin
    subtitle_y = title_y + int(H * 0.10)
    body_y = subtitle_y + int(H * 0.07)
    img_top = body_y + int(H * 0.12)

    draw_text_block(draw, title, TITLE_FONT, margin, title_y, W - 2*margin, subtitle_y)
    draw_text_block(draw, subtitle, SUBTITLE_FONT, margin, subtitle_y, W - 2*margin, body_y)
    draw_text_block(draw, body, BODY_FONT, margin, body_y, W - 2*margin, img_top)

    # 2×2 grid
    rows, cols = 2, 2
    cell_w = (W - 2*margin - (cols-1)*gutter) // cols
    cell_h = (H - img_top - margin - (rows-1)*gutter) // rows

    for i, img in enumerate(images[:4]):
        r, c = divmod(i, cols)
        x = margin + c * (cell_w + gutter)
        y = img_top + r * (cell_h + gutter)

        fitted = fit_image(img, cell_w, cell_h)
        fw, fh = fitted.size
        page.paste(fitted, (x + (cell_w-fw)//2, y + (cell_h-fh)//2))

    return page

In [ ]:
def layout_full_bleed(images, title, subtitle, body, page_size):
    W, H = page_size
    page = Image.new("RGB", page_size, WHITE)

    if images:
        bg = fit_image(images[0], W, H)
        bw, bh = bg.size
        page.paste(bg.crop(((bw-W)//2, (bh-H)//2, (bw+W)//2, (bh+H)//2)), (0, 0))

    overlay_h = int(H * 0.35)
    overlay = Image.new("RGBA", (W, overlay_h), (255, 255, 255, 235))
    page.paste(overlay, (0, H - overlay_h), overlay)

    draw = ImageDraw.Draw(page)
    margin = int(W * 0.06)

    title_y = H - overlay_h + margin
    subtitle_y = title_y + int(H * 0.08)
    body_y = subtitle_y + int(H * 0.06)

    draw_text_block(draw, title, TITLE_FONT, margin, title_y, W - 2*margin, subtitle_y)
    draw_text_block(draw, subtitle, SUBTITLE_FONT, margin, subtitle_y, W - 2*margin, body_y)
    draw_text_block(draw, body, BODY_FONT, margin, body_y, W - 2*margin, H - margin)

    return page

In [ ]:
def layout_image_led(images, title, subtitle, body, page_size):
    W, H = page_size
    page = Image.new("RGB", page_size, WHITE)
    draw = ImageDraw.Draw(page)

    margin = int(W * 0.06)
    gutter = int(W * 0.04)

    left_w = int(W * 0.58)
    right_x = margin + left_w + gutter

    # Left image
    if images:
        img = fit_image(images[0], left_w, H - 2*margin)
        page.paste(img, (margin, margin))

    # Right text
    title_y = margin
    subtitle_y = title_y + int(H * 0.12)
    body_y = subtitle_y + int(H * 0.08)

    draw_text_block(draw, title, TITLE_FONT, right_x, title_y, W - right_x - margin, subtitle_y)
    draw_text_block(draw, subtitle, SUBTITLE_FONT, right_x, subtitle_y, W - right_x - margin, body_y)
    draw_text_block(draw, body, BODY_FONT, right_x, body_y, W - right_x - margin, H - margin)

    return page

In [ ]:
def layout_text_led(images, title, subtitle, body, page_size):
    W, H = page_size
    page = Image.new("RGB", page_size, WHITE)
    draw = ImageDraw.Draw(page)

    margin = int(W * 0.06)
    gutter = int(W * 0.04)

    text_h = int(H * 0.45)

    title_y = margin
    subtitle_y = title_y + int(text_h * 0.25)
    body_y = subtitle_y + int(text_h * 0.25)
    img_top = margin + text_h + gutter

    draw_text_block(draw, title, TITLE_FONT, margin, title_y, W - 2*margin, subtitle_y)
    draw_text_block(draw, subtitle, SUBTITLE_FONT, margin, subtitle_y, W - 2*margin, body_y)
    draw_text_block(draw, body, BODY_FONT, margin, body_y, W - 2*margin, img_top)

    if images:
        img = fit_image(images[0], W - 2*margin, H - img_top - margin)
        page.paste(img, (margin, img_top))

    return page

In [ ]:
def build_page(images, title, subtitle, body, style, orientation):
    page_size = A4_LANDSCAPE if orientation == "Landscape" else A4_PORTRAIT

    style = style.lower()
    if style == "grid":
        return layout_grid(images, title, subtitle, body, page_size)
    if style == "full-bleed":
        return layout_full_bleed(images, title, subtitle, body, page_size)
    if style == "image-led":
        return layout_image_led(images, title, subtitle, body, page_size)
    if style == "text-led":
        return layout_text_led(images, title, subtitle, body, page_size)

    return layout_grid(images, title, subtitle, body, page_size)


def generate_spread(img1, img2, img3, img4, title, subtitle, body, style, orientation):
    images = [i for i in [img1, img2, img3, img4] if i]

    page = build_page(images, title, subtitle, body, style, orientation)

    tmp = tempfile.mkdtemp()
    pdf_path = os.path.join(tmp, "spread.pdf")
    page.convert("RGB").save(pdf_path, "PDF", resolution=300)

    return page, pdf_path

In [ ]:
with gr.Blocks(title="Portfolio Layout Generator") as demo:
    gr.Markdown("# Portfolio Layout Generator")

    with gr.Row():
        with gr.Column():
            title = gr.Textbox(label="Title")
            subtitle = gr.Textbox(label="Subtitle")
            body = gr.Textbox(label="Body text", lines=8)

            style = gr.Dropdown(
                ["Grid", "Full-bleed", "Image-led", "Text-led"],
                value="Grid",
                label="Layout style"
            )

            orientation = gr.Dropdown(
                ["Landscape", "Portrait"],
                value="Landscape",
                label="Orientation"
            )

            generate = gr.Button("Generate Spread")

        with gr.Column():
            img1 = gr.Image(label="Image 1", type="pil")
            img2 = gr.Image(label="Image 2", type="pil")
            img3 = gr.Image(label="Image 3", type="pil")
            img4 = gr.Image(label="Image 4", type="pil")

    preview = gr.Image(label="Preview", type="pil")
    pdf = gr.File(label="Download PDF")

    generate.click(
        generate_spread,
        inputs=[img1, img2, img3, img4, title, subtitle, body, style, orientation],
        outputs=[preview, pdf]
    )

demo.launch()

IMPORTANT: You are using gradio version 4.28.0, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e86b848e6d712711b8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
